In [1]:
from IPython.display import Markdown, display
from transistors import Transistor, NMOS, PMOS

<div style="text-align: center;">

## Targeted Requirements

<img src="two_stage_ota.png" alt="Two Stage OTA" width="500"/>

</div>


In [2]:
VSS: float = 0
VDD: float = 1.8
power_dissipated_max: float = 1 * 10**-3

slew_rate_min: float = 10 * 10**6
load_capacitance: float = 1 * 10**-15

open_circuit_gain_min: float = 5 * 10**3
ouput_voltage_range_target: float = 1
common_mode_voltage_input_min: float = 0.9
common_mode_voltage_input_max: float = 1.5
voltage_output_in_common_mode: float = VDD / 2

band_width_min: float = 5 * 10**6
phase_margin_min: float = 60

minimal_lenght: float = 0.18 * 10**-6
nominal_lenght: float = 10 * minimal_lenght

<div style="text-align: center;">

## Simulation Values

$$k_n^{'} = \mu_n \cdot C_{ox}$$

$$k_p^{'} = \mu_p \cdot C_{ox}$$

$$|V_{An}^{'}| = \frac{V_{An}}{L_{min}}$$

$$|V_{Ap}^{'}| = \frac{V_{Ap}}{L_{min}}$$

</div>


In [3]:
nmos_transconductance_parameter = 215 * 10**-6
nmos_early_voltage_slope = 30 / (3 * minimal_lenght) * 10**6

pmos_transconductance_parameter = 50 * 10**-6
pmos_early_voltage_slope = 16.5 / (3 * minimal_lenght)

nmos_threshold_voltage = 0.5
pmos_threshold_voltage = 0.5

<div style="text-align: center;">

## Transistor declaration

</div>


In [4]:
nmos_instances = []
pmos_instances = []
for i in range(5):
    nmos_instances.append(
        NMOS(
            transconductance_parameter=nmos_transconductance_parameter,
            threshold_voltage=nmos_threshold_voltage,
        )
    )
for i in range(3):
    pmos_instances.append(
        PMOS(
            transconductance_parameter=pmos_transconductance_parameter,
            threshold_voltage=pmos_threshold_voltage,
        )
    )

Q5, Q7, Q8, Q1, Q2 = nmos_instances
Q6, Q4, Q3 = pmos_instances

<div style="text-align: center;">

## Compensation Capacitor $C_C$

$$C_C = \frac{2.2}{10} \cdot C_{L}$$

</div>


In [5]:
compensation_capacitor = 2.2 / 10 * load_capacitance

display(Markdown(f"$C_{{C}} = {compensation_capacitor} \\text{{F}}$"))

$C_{C} = 2.2000000000000005e-16 \text{F}$

<div style="text-align: center;">

## Current in $Q_5$

$$I_{D5} = SR \cdot C_C$$

</div>


In [6]:
Q5.current = slew_rate_min * compensation_capacitor

display(Markdown(f"$I_{{D5}} = {Q5.current} \\text{{A}}$"))

$I_{D5} = 2.2000000000000007e-09 \text{A}$

<div style="text-align: center;">

## Curent in $Q_{1,2,3,4}$

$$I_{D1,2,3,4} = \frac{I_{D5}}{2}$$

</div>


In [7]:
for transistor in [Q1, Q2, Q3, Q4]:
    transistor.current = Q5.current / 2

display(Markdown(f"$I_{{D1,2,3,4}} = {transistor.current} \\text{{A}}$"))

$I_{D1,2,3,4} = 1.1000000000000003e-09 \text{A}$

<div style="text-align: center;">

## $V_{G1min}$

$$V_{G1min} = V_{CMmin} = 0.9 \text{V}$$

### Cool Title 1

$$V_{GD} \leq V_{tn}$$

$$V_{G} - V_{D} \leq V_{tn}$$

$$V_{G} \leq V_{D} + V_{tn}$$

### Cool Title 2

$$V_{D1} = V_{SS} + V_{OV5} + V_{DS1}$$

#### When $V_G$ is at its Minimum

$$V_{DS1} = V_{OV1}$$

$$V_{D1} = V_{SS} + V_{OV5} + V_{OV1}$$

### Cool Title 4

$$V_{G1min} = V_{SS} + V_{OV5} + V_{OV1} + V_{tn}$$

</div>


In [8]:
# relevant python code here

<div style="text-align: center;">

## Transistor $Q_3$

$$V_{G1max} = V_{CMmax} = 1.5 \text{V}$$

### Useful Identity A : Gate to Drain Voltage Requirement for Saturation

$$V_{GD} \leq V_{tn} $$

$$V_{G} - V_{D} \leq V_{tn}$$

$$V_{G} \leq V_{D} + V_{tn}$$

### Useful Identity B : Fiding an Equation the Drain Voltage of $Q_1$

$$V_{D1} = V_{DD} - V_{SG3}$$

### Identity B into A

$$V_{G1max} = V_{DD} - V_{SG3} + V_{tn}$$

$$V_{SG3} = V_{DD} - V_{G1max} + V_{tn}$$

</div>


In [9]:
Q3.voltage_source_gate = VDD - common_mode_voltage_input_max + Q3.threshold_voltage

display(Markdown(f"$V_{{SG3}} = {Q3.voltage_source_gate} \\text{{V}}$"))

$V_{SG3} = 0.8 \text{V}$

<div style="text-align: center;">

## Overdrive Voltage for $Q_3$

$V_{OV3} = V_{SG3} - V_{t3}$

</div>


In [10]:
Q3.overdrive_voltage = Q3.voltage_source_gate - Q3.threshold_voltage

display(Markdown(f"$V_{{OV3}} = {Q3.overdrive_voltage} \\text{{V}}$"))

$V_{OV3} = 0.30000000000000004 \text{V}$

<div style="text-align: center;">

## Isolating of the Width in the Current Equation

### With channel lenght modulation

$W = \frac{2 I_{D} \cdot L}{k_{x}^{'} \cdot V_{OV}^2 (1 + \frac{V_{DS}}{|V_{Ax}|})}$

### Without Channel Lenght Modulation

$W = \frac{2 I_{D} \cdot L}{k_{x}^{'} \cdot V_{OV}^2}$

</div>


In [11]:
# add relevant code here (transistor.py functions ?)

<div style="text-align: center;">

## $v_o$ range

$$v_o = I_{D6} \cdot (r_{o7} // r_{o6})$$

</div>


In [12]:
# put relevant code here

<div style="text-align: center;">

## Open-loop Gain ($A_{v}$)

$A_{v} = g_{m6} \cdot g_{m2} \cdot (r_{o2}//r_{o4}) \cdot (r_{o6}//r_{o7})$

</div>


In [13]:
Av = (
    Q6.transconductance_parameter
    * Q2.transconductance_parameter
    * ((1 / (1 / Q2.output_resistance)) + (1 / (1 / Q4.output_resistance)))
    * ((1 / (1 / Q7.output_resistance)) + (1 / (1 / Q6.output_resistance)))
)

TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'